# Map of all Gas stations in Germany

In [2]:
# Import libraries

import pandas as pd
import numpy as np
import plotly as plt
import plotly_express as px
import plotly.graph_objects as go

In [4]:
# dataframe
stations = pd.read_csv('../data/2025-05-25-stations.csv')
stations.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,first_active,openingtimes_json
0,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,OIL!,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":192,""perio..."
1,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752009,9.279039,1970-01-01 01:00:00+01,{}
2,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760,1970-01-01 01:00:00+01,"{""overrides"":[{""startp"":""2025-05-29 06:00"",""en..."
3,cdaa1ef5-9c3d-499d-869a-1c970beba775,OIL! tank &amp; go Automatentankstelle Friedri...,OIL! (Automatenstation),Koogstr. 16,NaN,25718,Friedrichskoog,53.992100,8.940690,2014-03-18 16:45:31+01,{}
4,005056ba-7cb6-1ed2-bceb-66e14a634d1f,ORLEN Tankstelle,ORLEN,Curt-Schröter-Straße,2,39179,Barleben / Ebendorf,52.181182,11.585476,2014-03-18 16:45:31+01,{}


In [5]:
#we delete the stations that start with 00000, which are just test data
stations = stations[~stations["uuid"].astype(str).str.startswith("000000")]

In [6]:
# replace the missing name of Günstige Tankstelle Schaal with the brand name
stations.loc[stations['brand'] == 'Günstige Tankstelle Schaal', 'name'] = stations['brand']

In [7]:
# delete deleted entries
stations= stations[~stations['name'].str.lower().str.contains('gelöscht|please delete', na=False)]

In [8]:
stations.brand.nunique()

1152

In [9]:
known_brands=['aral', 'shell', 'esso', 'total', 'avia', 'jet', 'star', 'agip eni', 'raiffeisen', 'bft', 'oil!', 'sb']

In [10]:
stations_clean = stations.copy()

In [11]:
def extract_brand(text):
    text = str(text).lower()
    for brand in known_brands:
        if brand in text:
            return brand
    return None

# Create 'brand_clean' column
def get_clean_brand(row):
    # First try from brand column
    brand_value = extract_brand(row["brand"])
    if brand_value:
        return brand_value
    
    # Then try from name column
    name_value = extract_brand(row["name"])
    if name_value:
        return name_value
    
    # Otherwise: unknown
    return "other"

In [12]:
# Apply to DataFrame
stations_clean["brand_clean"] = stations_clean.apply(get_clean_brand, axis=1)

In [13]:
brand_colors = {
    'aral': '#0069B3',
    'shell': '#FFD100',
    'total': '#EA1C24',
    'esso': '#003399',
    'avia': '#333333',
    'bft' : '#EF4023',
    'jet' : '#FFDD00',
    'sb' : '#999999',
    'raiffeisen' : '#00A651',
    'star' : '#E50010',
    'agip eni' : '#FFCC00',
    'oil!' : '#212121',
    'freie tankstelle' : '',
    'other': "#BBAEAE"
}

In [14]:
colors = stations_clean['brand_clean'].map(brand_colors)

In [15]:
map =   px.scatter_map(
        stations_clean, 
        lat = 'latitude',
        lon = 'longitude',
        hover_name = 'name',
        hover_data = {'latitude' : False, 'longitude' : False},
        #center = {'lat': 51.1634, 'lon': 10.4477},
        zoom = 5.4,
        color='brand_clean',
        color_discrete_map=brand_colors,
        labels={"brand_clean": "brand"}
        )
map.update_layout(autosize=False,
    width=1200,
    height=800,
    margin={
        'l':50,
        'r':50,
        'b':30,
        't':30},
    mapbox_center = {'lat': 51.1634, 'lon': 10.4477} 
)
map.show()

In [15]:
heatmap = px.density_map(stations, lat='latitude', 
                         lon ='longitude',  
                         radius = 5,
                         center = {'lat': 51.1634, 'lon': 10.4477}, 
                         zoom = 5,
                         #map_style="open-street-map"
)
heatmap.update_layout(autosize=False,
    width=1200,
    height=800,
    margin={
        'l':50,
        'r':50,
        'b':30,
        't':30}
)
heatmap.show()

In [16]:

# Rübenkamp 269 Koordinaten Lat: 53.6097731, Lon: 10.0330959

Mylat = 53.6097731
Mylon= 10.0330959
Myradius = 5

In [17]:


from get_current_fuel_prices import get_current_fuel_prices

Location = get_current_fuel_prices(Mylat, Mylon, Myradius)

Location.head()

,id,name,brand,street,place,lat,lng,dist,diesel,e5,e10,isOpen,houseNumber,postCode
0,512f9ee3-77cf-4719-f51a-b837c985f035,CleanCar AG NL123,CleanCar,Steilshooper Allee,Hamburg,53.608500,10.045570,0.8,1.609,1.819,1.759,True,5,22309
1,cd8ba6a6-8316-1ed5-a3ae-d3139800d85f,JET HAMBURG STEILSHOOPER ALLEE 9,JET,STEILSHOOPER ALLEE,HAMBURG,53.608580,10.046870,0.9,1.649,1.819,1.759,True,9,22309
2,462bfd6e-ae5f-4da8-acfa-6e88cf503d1b,Esso Tankstelle,ESSO,UEBERSEERING 1,HAMBURG,53.601260,10.024281,1.1,1.669,1.829,1.769,True,,22297
3,cfaf5e3c-60ee-4a11-a6c5-5b359a67dded,Shell Hamburg Steilshooper Allee 55,Shell,Steilshooper Allee,Hamburg,53.608899,10.051451,1.2,1.649,1.829,1.769,True,55,22309
4,51d4b540-a095-1aa0-e100-80009459e03a,JET HAMBURG STEILSHOOPER ALLEE 44,JET,STEILSHOOPER ALLEE,HAMBURG,53.608260,10.056240,1.5,1.649,1.819,1.759,True,44,22309


In [18]:
# function that calculates the edge coordinates of the circle given a circle radius in km

def make_circle(lat, lon, radius_km, num_points=100):
    earth_radius = 6371  # in km
    lats, lons = [], []
    for i in range(num_points):
        angle = 2 * np.pi * i / num_points
        dx = radius_km * np.cos(angle)
        dy = radius_km * np.sin(angle)
        delta_lat = (dy / earth_radius) * (180 / np.pi)
        delta_lon = (dx / (earth_radius * np.cos(np.pi * lat / 180))) * (180 / np.pi)
        lats.append(lat + delta_lat)
        lons.append(lon + delta_lon)
    lats.append(lats[0])
    lons.append(lons[0])

    return lats, lons

In [19]:
circle_lats, circle_lons = make_circle(Mylat, Mylon, Myradius)

In [20]:
circle_lats, circle_lons = make_circle(Mylat, Mylon, Myradius)
pricemap =   px.scatter_map(
        Location, 
        lat = 'lat',
        lon = 'lng',
        hover_name = 'name',
        hover_data = {'lat' : False, 'lng' : False, 'e5' : True, 'e10' : True, 'diesel' : True, 'isOpen' : True},
        center = {'lat': Mylat, 'lon': Mylon},
        zoom = 11,
        )
pricemap.update_layout(autosize=False,
    width=1200,
    height=800,
    margin={
        'l':50,
        'r':50,
        'b':30,
        't':30}
)
pricemap.add_trace(
    go.Scattermap(
        lat=circle_lats,
        lon=circle_lons,
        mode='lines',
        fill='toself',
        fillcolor='rgba(0, 100, 255, 0.1)',
        line=dict(color='blue'),
        name=f"{Myradius} km radius"
    )
)
pricemap.show()

In [21]:
# number of stations per postcode per brand
plz = stations_clean.groupby(['post_code', 'brand_clean']).count().reset_index()
plz

,post_code,brand_clean,uuid,name,brand,street,house_number,city,latitude,longitude,first_active,openingtimes_json
0,01067,total,1,1,1,1,1,1,1,1,1,1
1,01069,total,1,1,1,1,1,1,1,1,1,1
2,01097,esso,1,1,1,1,1,1,1,1,1,1
3,01097,other,1,1,1,1,1,1,1,1,1,1
4,01097,shell,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13158,99994,agip eni,1,1,1,1,1,1,1,1,1,1
13159,99994,other,1,1,1,1,1,1,1,1,1,1
13160,99996,other,1,1,1,1,1,1,1,1,1,1
13161,Nicht,other,1,1,1,1,0,1,1,1,1,1


In [22]:
# dominant brand per postcode
idx = plz.groupby('post_code')['uuid'].idxmax()
dominant_brands = plz.loc[idx].reset_index(drop=True)

In [23]:
brand_colors = {
    'aral': '#0069B3',
    'shell': '#FFD100',
    'total': '#EA1C24',
    'esso': '#003399',
    'avia': '#333333',
    'bft' : '#EF4023',
    'jet' : '#FFDD00',
    'sb' : '#999999',
    'raiffeisen' : '#00A651',
    'star' : '#E50010',
    'agip eni' : '#FFCC00',
    'oil!' : '#212121',
    'freie tankstelle' : '',
    'other': "#BBAEAE"
}

In [24]:
colors = dominant_brands['brand_clean'].map(brand_colors)

In [25]:
geojson_data='data/postleitzahlen.geojson'

brands = px.choropleth_map(
    dominant_brands,
    geojson=geojson_data,
    locations="post_code",
    featureidkey="properties.postcode",
    color_discrete_map=brand_colors,
    opacity=0.65,
    color="brand_clean",
    center = {'lat': 51.1634, 'lon': 10.4477}, # center of Germany
    zoom = 5,  # Adjust this value to zoom in/out
)


brands.update_layout(
    title={
        'text': "Dominant Brands per Postcode",
        'x': 0.5,  # Center the title (0 = left, 0.5 = center, 1 = right)
        'xanchor': 'center'
    },
    autosize=False,
    width=1400,
    height=600,
     coloraxis_colorbar=dict(
        title="Brands",     # <br> adds a line break (optional)
        #title_side="right",           # this puts it to the right of the bar (default)
        title_font=dict(size=14),     # font size
        ),                        
    margin=dict(
        l=50,
        r=50,
        b=30,
        t=30
    )
)
brands.show()